## Search Engine With Tools And Agents

In [45]:
import os
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper
from langchain_classic.tools.retriever import create_retriever_tool
from langchain_classic.agents import create_openai_tools_agent, AgentExecutor
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv 

from langchain_groq import ChatGroq

from langchain_classic import hub

In [33]:
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

In [17]:
# Used the inbuilt tool of wikipedia
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki_tool=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)

In [18]:
# Used the inbuilt tool of arxiv
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv_tool = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)

In [19]:
# Custom tool [RAG tool]
loader = WebBaseLoader("https://docs.langchain.com/langsmith/home")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents,HuggingFaceEmbeddings())
retriever = vectordb.as_retriever()

## create retriever as tool
retriever_tool = create_retriever_tool(retriever, "langsmith=search", "search any information about Langsmith")


In [20]:
tools = [wiki_tool, arxiv_tool, retriever_tool]

In [35]:
## Run all this tools with Agents and LLM Models
llm = ChatGroq(groq_api_key=groq_api_key, model_name="llama-3.3-70b-versatile")

In [ ]:
## Prompt Template
# An alternate way for prompt template
prompt = hub.pull('hwchase17/openai-functions-agent')
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [ ]:
## Agents
agent = create_openai_tools_agent(llm,tools, prompt)

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [47]:
# Agent Executer
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)

In [48]:
agent_executor.invoke({"input": "Tell me about Langsmith"})




> Entering new AgentExecutor chain...

Invoking: `langsmith=search` with `{'query': 'Langsmith information'}`


LangSmith docs - Docs by LangChainSkip to main contentDocs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationLangSmith docsGet startedObservabilityEvaluationPrompt engineeringDeploymentAgent BuilderPlatform setupReferenceOverviewPlansCreate an account and API keyAccount administrationOverviewSet up hierarchyManage organizations using the APIManage billingSet up resource tagsUser managementAdditional resourcesPolly (Beta)Data managementAccess control & AuthenticationScalability & resilienceFAQsRegions FAQPricing FAQLangSmith statusLangSmith docsCopy pageCopy pageLangSmith provides tools for developing, debugging, and deploying LLM applications.
It helps you trace requests, evaluate outputs, test prompts, and manage deployments in one place.
LangSmith is framework agnostic, so you can use it with or without LangChain’s open-sou

{'input': 'Tell me about Langsmith',
 'output': "LangSmith is a platform that provides tools for developing, debugging, and deploying Large Language Model (LLM) applications. It offers features such as observability, evaluation, prompt testing, and deployment, and is framework agnostic, allowing users to use it with or without LangChain's open-source libraries. LangSmith provides a visual interface for designing and deploying AI agents, as well as a no-code interface for rapid prototyping. It also meets high standards of data security and privacy, with HIPAA, SOC 2 Type 2, and GDPR compliance."}

In [50]:
agent_executor.invoke({"input": "What is machine learning"})



> Entering new AgentExecutor chain...


APIError: Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.